# HN Rerank - Fine-tune Embeddings

Run this notebook in Google Colab with GPU runtime.

**Setup:** Runtime → Change runtime type → T4 GPU

In [ ]:
# 1. Install dependencies
!pip install -q sentence-transformers torch

In [ ]:
# 2. Upload training data
from google.colab import files
print("Upload train_triplets.jsonl and val_triplets.jsonl")
uploaded = files.upload()

In [ ]:
# 3. Verify GPU
import torch
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# 4. Training script
import json
from pathlib import Path
from sentence_transformers import SentenceTransformer, losses, InputExample
from sentence_transformers.evaluation import TripletEvaluator
from torch.utils.data import DataLoader

def load_triplets(path):
    examples = []
    for line in Path(path).read_text().strip().split("\n"):
        if not line:
            continue
        item = json.loads(line)
        examples.append(InputExample(texts=[item["anchor"], item["positive"], item["negative"]]))
    return examples

# Load data
train_examples = load_triplets("train_triplets.jsonl")
val_examples = load_triplets("val_triplets.jsonl") if Path("val_triplets.jsonl").exists() else []

print(f"Training examples: {len(train_examples)}")
print(f"Validation examples: {len(val_examples)}")

In [ ]:
# 5. Load model and train
model = SentenceTransformer("BAAI/bge-base-en-v1.5")

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
train_loss = losses.TripletLoss(
    model=model,
    distance_metric=losses.TripletDistanceMetric.COSINE,
    triplet_margin=0.5
)

# Evaluator
evaluator = None
if val_examples:
    evaluator = TripletEvaluator(
        [t.texts[0] for t in val_examples],
        [t.texts[1] for t in val_examples],
        [t.texts[2] for t in val_examples],
        name="hn_eval"
    )

# Train
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=3,
    warmup_steps=10,
    output_path="tuned_model",
    show_progress_bar=True
)

print("Training complete!")

In [ ]:
# 6. Export to ONNX
!pip install -q optimum onnx onnxruntime

!optimum-cli export onnx --model tuned_model --task feature-extraction onnx_model

print("ONNX export complete!")

In [ ]:
# 7. Download the model
import shutil
shutil.make_archive("onnx_model", "zip", "onnx_model")
files.download("onnx_model.zip")

print("Download complete! Extract to hn_rerank/onnx_model/")